# Overview

### 3 main objects:

1. Assistant - this is the LLM that has access to 'tools'. 
2. Thread - This is the the 'chat window' where the user/assistant carry out their conversation through back and forth 'messages'. This reminds me of dicussion forums of old - where you can create 'threads' to start a discussion. 
3. Messages - the 'message' of the user or assistant added to the thread. The assistant 'message/s' are only generated during 'Runs'.

### Runs

After creating the assistant, thread and 'initial' user message e.g. 'Hello!', you create a 'Run'.

During Runs, the Assistant evaluates all the existing 'messages' in the thread and adds a 'message' to the thread accordingly e.g. assistant message: 'Hi, how can I assist you today?'

Part of the Assistants job is to determine if a tool is required to be used given the existing 'messages' in the 'thread'. The above user message ('Hello!'), does not require a tool, for example. If the next user message is 'Hows the weather today in Manila?', the next Run should have Assistant use the 'get_weather' tool.

Each user message and run is created in succession to maintain the ongoing conversation using the following:


- client.beta.threads.messages.create() - function/endpoint for creating the User Messages

- client.beta.threads.runs.create() - function/endpoint for creating Runs







# Scenarios Covered

1. A user message does not require the Assistant to use a 'tool'.
2. A user message requires the Assistant to use a 'tool'.
3. A user message requires the Assistant to use multiple 'tools'.

### Imports

In [1]:
import os
import json
from dotenv import load_dotenv
from utils.assistants_api_object_pretty_printer import pretty_print, pretty_print_thread

load_dotenv()


openai_api_key = os.getenv('OPENAI_API_KEY')

from openai import OpenAI
client = OpenAI()


## Step 1: Create Assistant

Assistants should be created once and referenced all throught via the assistant.id

This is can also be created in the playground UI: https://platform.openai.com/playground


In [4]:
from openai import OpenAI
client = OpenAI()

assistant = client.beta.assistants.create(
    instructions="You are a friendly assistant.",
    name="Test Assistant",    
    model="gpt-3.5-turbo-1106",    
    tools = [
    {
            "type": "function",
            "function": {
                "name": "getCurrentWeather",
                "description": "Get the weather in location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string", 
                            "description": "The city and state e.g. San Francisco, CA"
                        },
                        "unit": {
                            "type": "string", 
                            "enum": ["c", "f"]
                        }
                    },
                    "required": ["location"]
                }
            },
        },
    {
            "type": "function",
            "function": {
                "name": "getCurrentTraffic",
                "description": "Get the current traffic in location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string", 
                            "description": "The city and state e.g. San Francisco, CA"
                        }                        
                    },
                    "required": ["location"]
                }
            },
        }
  ]
)
pretty_print(assistant)

{
  "id": "asst_GM37mvTmfH0zWAWe4BEuLtGn",
  "created_at": "1702014088",
  "description": null,
  "file_ids": [],
  "instructions": "You are a friendly assistant.",
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "name": "Test Assistant",
  "object": "assistant",
  "tools": [
    {
      "function": {
        "name": "getCurrentWeather",
        "parameters": "{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}",
        "description": "Get the weather in location"
      },
      "type": "function"
    },
    {
      "function": {
        "name": "getCurrentTraffic",
        "parameters": "{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}}, 'required': ['location']}",
        "description": "Get the current traffic in location"
      },
      "type": "functio

### Utils: View Details of Existing Assistant

In [2]:
assistant= client.beta.assistants.retrieve("asst_GM37mvTmfH0zWAWe4BEuLtGn")
pretty_print(assistant)

{
  "id": "asst_GM37mvTmfH0zWAWe4BEuLtGn",
  "created_at": "1702014088",
  "description": null,
  "file_ids": [],
  "instructions": "You are a friendly assistant.",
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "name": "Test Assistant",
  "object": "assistant",
  "tools": [
    {
      "function": {
        "name": "getCurrentWeather",
        "parameters": "{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}",
        "description": "Get the weather in location"
      },
      "type": "function"
    },
    {
      "function": {
        "name": "getCurrentTraffic",
        "parameters": "{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}}, 'required': ['location']}",
        "description": "Get the current traffic in location"
      },
      "type": "functio

### Utils: Update assistant (for example if you want to add/remove tools)

In this example, I added an additional 'tool' that can be used by the Assistant - the 'Code Interpreter'

In [3]:
assistant = client.beta.assistants.update(
    instructions="You are a friendly assistant.",
    name="Test Assistant",
    assistant_id = assistant.id,
    model="gpt-3.5-turbo-1106",    
    tools = [
    {
            "type": "function",
            "function": {
                "name": "getCurrentWeather",
                "description": "Get the weather in location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string", 
                            "description": "The city and state e.g. San Francisco, CA"
                        },
                        "unit": {
                            "type": "string", 
                            "enum": ["c", "f"]
                        }
                    },
                    "required": ["location"]
                }
            },
        },
    {
            "type": "function",
            "function": {
                "name": "getCurrentTraffic",
                "description": "Get the current traffic in location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string", 
                            "description": "The city and state e.g. San Francisco, CA"
                        }                        
                    },
                    "required": ["location"]
                }
            },
        },
    {
            "type": "code_interpreter",            
        }
  ]
)
pretty_print(assistant)


{
  "id": "asst_GM37mvTmfH0zWAWe4BEuLtGn",
  "created_at": "1702014088",
  "description": null,
  "file_ids": [],
  "instructions": "You are a friendly assistant.",
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "name": "Test Assistant",
  "object": "assistant",
  "tools": [
    {
      "function": {
        "name": "getCurrentWeather",
        "parameters": "{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}",
        "description": "Get the weather in location"
      },
      "type": "function"
    },
    {
      "function": {
        "name": "getCurrentTraffic",
        "parameters": "{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}}, 'required': ['location']}",
        "description": "Get the current traffic in location"
      },
      "type": "functio

# Step 2: Create Thread

TIP: Threads can be assigned to different assistants (.e.g. if you want to evaluate different models, you can create the same exact assistant but one powered by GPT3.5, another by GPT4)

In [4]:
thread = client.beta.threads.create()
pretty_print(thread)

{
  "id": "thread_1nr34IXRHNQOipkQODjmrIUN",
  "created_at": "1702018057",
  "metadata": "{}",
  "object": "thread"
}


# Scenario 1: A user message does not require the Assistant to use a 'tool'.

## Step 3: Create Message

User message: 'Hi Assistant!'

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Hi Assistant!"
)
pretty_print(message)

{
  "id": "msg_yKl0nWEuokUUIBmkZyyYFfzp",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "Hi Assistant!"
      },
      "type": "text"
    }
  ],
  "created_at": "1702018060",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN"
}


### Utils: Display Existing Messages in Thread ID

In [6]:
thread_messages = client.beta.threads.messages.list(thread.id)

for index in range(len(thread_messages.data)):
    pretty_print(thread_messages.data[index])

{
  "id": "msg_yKl0nWEuokUUIBmkZyyYFfzp",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "Hi Assistant!"
      },
      "type": "text"
    }
  ],
  "created_at": "1702018060",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN"
}


## Step 4: Create Run

During Runs, the assistant evaluates the content of the thread and add message/s as necessary.

In [7]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id  
)

## Step 5: Wait for Run to finish then View the Thread

The assistant should add a 'message' to the thread - "Hello! How can I assist you today?"

In [8]:
thread_messages = client.beta.threads.messages.list(thread.id)

for index in range(len(thread_messages.data)):
    pretty_print(thread_messages.data[index])

{
  "id": "msg_xxHLiMEdSREOSUesvkRCiuFo",
  "assistant_id": "asst_GM37mvTmfH0zWAWe4BEuLtGn",
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "Hello! How can I assist you today?"
      },
      "type": "text"
    }
  ],
  "created_at": "1702018073",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "assistant",
  "run_id": "run_Y0hFSlf96zIu1tIRXxupPXCY",
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN"
}
{
  "id": "msg_yKl0nWEuokUUIBmkZyyYFfzp",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "Hi Assistant!"
      },
      "type": "text"
    }
  ],
  "created_at": "1702018060",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN"
}


# Scenario 2:  A user message requires the Assistant to use a 'tool'.

## Step 6: Create Message

User Message: 'How's the weather in Manila today?'

In [9]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="How's the weather in Manila today?"
)
pretty_print(message)

{
  "id": "msg_TnEeySsXrcgEVqZkQzCVDPC8",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "How's the weather in Manila today?"
      },
      "type": "text"
    }
  ],
  "created_at": "1702018090",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN"
}


## Step 7: Create Run 2

In [10]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id)

For user messages that does not require 'tool' use, the Run status is set to 'completed' and 'assistant' message is added to the thread.

For user messages that requires 'tool' use, an intermediate Run status called 'requires_action' is set.

Note: Other Run status values can be: 
- queued 
- in_progress
- requires_action
- cancelling
- cancelled 
- failed
- completed
- expired

## Step 8: Wait for Run to finish then View status of Run

Note the value of run.status = 'requires_action'.

The run.required_action value also indicates that the assistant 'decided' to use the "getCurrentWeather" tool. 

It also identified the arguments ('Manila', 'celsius') that should be used with the tool/function to get the requested info (weather in Manila) by the user.

In [11]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

run = wait_on_run(run, thread)
pretty_print(run)


{
  "id": "run_EtEZoN5CF0vkonwUFDvqEWob",
  "assistant_id": "asst_GM37mvTmfH0zWAWe4BEuLtGn",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": "1702018104",
  "expires_at": "1702018704",
  "failed_at": null,
  "file_ids": [],
  "instructions": "You are a friendly assistant.",
  "last_error": null,
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "object": "thread.run",
  "required_action": {
    "submit_tool_outputs": {
      "tool_calls": [
        {
          "id": "call_dpTcq4iks2jGcdvo42pCnoh0",
          "function": {
            "arguments": "{'location': 'Manila', 'unit': 'c'}",
            "name": "getCurrentWeather"
          },
          "type": "function"
        }
      ]
    },
    "type": "submit_tool_outputs"
  },
  "started_at": "1702018104",
  "status": "requires_action",
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN",
  "tools": [
    {
      "function": {
        "name": "getCurrentWeather",
        "parameters": "{'type': 'object', 'properties'

# If status = requires_action: 

This status means the assistant wanted to use a function call and has outputted the function name and arguments to use for the function.

### Initialize tool_call as var 

It's possible that it will have several index/page of tools to use based on the nature of the query

This way, we can just iterate on this object by calling the index position


In [12]:
tool_call = run.required_action.submit_tool_outputs.tool_calls
pretty_print(tool_call)

[
  {
    "id": "call_dpTcq4iks2jGcdvo42pCnoh0",
    "function": {
      "arguments": "{'location': 'Manila', 'unit': 'c'}",
      "name": "getCurrentWeather"
    },
    "type": "function"
  }
]



## Step 9: Update Run with Tool Output

The 'getCurrentWeather' function should be executed ~elsewhere in your program~ using the args provided by the assistant.

In this example, the output of getCurrentWeather("Manila", "celsius") is '19C'.

The run is updated with the output of the tool/s

In [13]:
run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=[
    {
      "tool_call_id": tool_call[0].id,
      "output": "19C"
    }
  ]
)
pretty_print(run)

{
  "id": "run_EtEZoN5CF0vkonwUFDvqEWob",
  "assistant_id": "asst_GM37mvTmfH0zWAWe4BEuLtGn",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": "1702018104",
  "expires_at": "1702018704",
  "failed_at": null,
  "file_ids": [],
  "instructions": "You are a friendly assistant.",
  "last_error": null,
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "object": "thread.run",
  "required_action": null,
  "started_at": "1702018104",
  "status": "queued",
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN",
  "tools": [
    {
      "function": {
        "name": "getCurrentWeather",
        "parameters": "{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}",
        "description": "Get the weather in location"
      },
      "type": "function"
    },
    {
      "function": {
        "name": "getCurrentTraffic",
        "parameters"

## Step 10: Wait for Run to finish then View the Thread

The assistant should add the message after synthesing the output of the tool: "The weather in Manila today is 19\u00b0C."

In [14]:
thread_messages = client.beta.threads.messages.list(thread.id)

for index in range(len(thread_messages.data)):
    pretty_print(thread_messages.data[index])

{
  "id": "msg_UIFj75kmx2ULBG0DLEu0ySCV",
  "assistant_id": "asst_GM37mvTmfH0zWAWe4BEuLtGn",
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "The weather in Manila today is 19\u00b0C."
      },
      "type": "text"
    }
  ],
  "created_at": "1702018122",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "assistant",
  "run_id": "run_EtEZoN5CF0vkonwUFDvqEWob",
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN"
}
{
  "id": "msg_TnEeySsXrcgEVqZkQzCVDPC8",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "How's the weather in Manila today?"
      },
      "type": "text"
    }
  ],
  "created_at": "1702018090",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN"
}
{
  "id": "msg_xxHLiMEdSREOSUesvkRCiuFo",
  "assistant_id": "asst_GM37mvTmfH0zWAWe4BEuLtGn",
  "content": [
    {
 

### Utils: Pretty Print Current Thread Content



In [16]:
print(pretty_print_thread(thread_messages))

User: "Hi Assistant!"
Assistant: "Hello! How can I assist you today?"
User: "How's the weather in Manila today?"
Assistant: "The weather in Manila today is 19°C."



# Scenario 3: A user message require the Assistant to use multiple 'tools'.

## Step 11: Create New Message

User message: "How's the weather and traffic condition in Cebu today?"

In [17]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="How's the weather and traffic condition in Cebu today?"
)
pretty_print(message)

{
  "id": "msg_hQPfBdODrGEDKOh0WKZrPx13",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "How's the weather and traffic condition in Cebu today?"
      },
      "type": "text"
    }
  ],
  "created_at": "1702018871",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN"
}


### Utils: Display Existing Messages in Thread ID

In [18]:
thread_messages = client.beta.threads.messages.list(thread.id)

for index in range(len(thread_messages.data)):
    pretty_print(thread_messages.data[index])

{
  "id": "msg_hQPfBdODrGEDKOh0WKZrPx13",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "How's the weather and traffic condition in Cebu today?"
      },
      "type": "text"
    }
  ],
  "created_at": "1702018871",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN"
}
{
  "id": "msg_UIFj75kmx2ULBG0DLEu0ySCV",
  "assistant_id": "asst_GM37mvTmfH0zWAWe4BEuLtGn",
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "The weather in Manila today is 19\u00b0C."
      },
      "type": "text"
    }
  ],
  "created_at": "1702018122",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "assistant",
  "run_id": "run_EtEZoN5CF0vkonwUFDvqEWob",
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN"
}
{
  "id": "msg_TnEeySsXrcgEVqZkQzCVDPC8",
  "assistant_id": null,
  "content": [
    {
      "t

## Step 12: Create Run

In [19]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id  
)

## Step 13: Wait for Run to finish then View

Note that the value of run.required_action indicates that the assistant decided to use 2 tools: 
- "getCurrentWeather" 
- "getCurrentTraffic". 

The arguments to use with these tools/function calls has been provided as well.

In [20]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

run = wait_on_run(run, thread)
pretty_print(run)

{
  "id": "run_KpXw8jkgyubwYwa1Tpjw2KVt",
  "assistant_id": "asst_GM37mvTmfH0zWAWe4BEuLtGn",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": "1702018899",
  "expires_at": "1702019499",
  "failed_at": null,
  "file_ids": [],
  "instructions": "You are a friendly assistant.",
  "last_error": null,
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "object": "thread.run",
  "required_action": {
    "submit_tool_outputs": {
      "tool_calls": [
        {
          "id": "call_iaAiy3aq57J9LIJQEQsvVEt1",
          "function": {
            "arguments": "{'location': 'Cebu', 'unit': 'c'}",
            "name": "getCurrentWeather"
          },
          "type": "function"
        },
        {
          "id": "call_ylkyAiSrLmts6Y7iihaNXKBO",
          "function": {
            "arguments": "{'location': 'Cebu'}",
            "name": "getCurrentTraffic"
          },
          "type": "function"
        }
      ]
    },
    "type": "submit_tool_outputs"
  },
  "started_at": "1

# If status = requires_action:

### Initialize tool_call as var 

In [21]:
tool_call = run.required_action.submit_tool_outputs.tool_calls
pretty_print(tool_call)

[
  {
    "id": "call_iaAiy3aq57J9LIJQEQsvVEt1",
    "function": {
      "arguments": "{'location': 'Cebu', 'unit': 'c'}",
      "name": "getCurrentWeather"
    },
    "type": "function"
  },
  {
    "id": "call_ylkyAiSrLmts6Y7iihaNXKBO",
    "function": {
      "arguments": "{'location': 'Cebu'}",
      "name": "getCurrentTraffic"
    },
    "type": "function"
  }
]


## Step 14: Update Run with Tool Output

We have simulated the following output for the respective function calls:

- "getCurrentWeather" - output: '28c' 
- "getCurrentTraffic" - output: 'carmageddon levels per the usual'



In [22]:
run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=[
    {
      "tool_call_id": tool_call[0].id,
      "output": "28C"
    },
    {
      "tool_call_id": tool_call[1].id,
      "output": "carmageddon levels per the usual"
    }

  ]
)
pretty_print(run)

{
  "id": "run_KpXw8jkgyubwYwa1Tpjw2KVt",
  "assistant_id": "asst_GM37mvTmfH0zWAWe4BEuLtGn",
  "cancelled_at": null,
  "completed_at": null,
  "created_at": "1702018899",
  "expires_at": "1702019499",
  "failed_at": null,
  "file_ids": [],
  "instructions": "You are a friendly assistant.",
  "last_error": null,
  "metadata": "{}",
  "model": "gpt-3.5-turbo-1106",
  "object": "thread.run",
  "required_action": null,
  "started_at": "1702018899",
  "status": "queued",
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN",
  "tools": [
    {
      "function": {
        "name": "getCurrentWeather",
        "parameters": "{'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The city and state e.g. San Francisco, CA'}, 'unit': {'type': 'string', 'enum': ['c', 'f']}}, 'required': ['location']}",
        "description": "Get the weather in location"
      },
      "type": "function"
    },
    {
      "function": {
        "name": "getCurrentTraffic",
        "parameters"

## Step 15: Display Content of Thread again

In [24]:
thread_messages = client.beta.threads.messages.list(thread.id)

for index in range(len(thread_messages.data)):
    pretty_print(thread_messages.data[index])

{
  "id": "msg_4RXAE87pt1wt5LhWspFJkdfx",
  "assistant_id": "asst_GM37mvTmfH0zWAWe4BEuLtGn",
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "The weather in Cebu today is 28\u00b0C, and the traffic condition is described as \"carmageddon levels per the usual.\""
      },
      "type": "text"
    }
  ],
  "created_at": "1702018935",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "assistant",
  "run_id": "run_KpXw8jkgyubwYwa1Tpjw2KVt",
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN"
}
{
  "id": "msg_hQPfBdODrGEDKOh0WKZrPx13",
  "assistant_id": null,
  "content": [
    {
      "text": {
        "annotations": [],
        "value": "How's the weather and traffic condition in Cebu today?"
      },
      "type": "text"
    }
  ],
  "created_at": "1702018871",
  "file_ids": [],
  "metadata": "{}",
  "object": "thread.message",
  "role": "user",
  "run_id": null,
  "thread_id": "thread_1nr34IXRHNQOipkQODjmrIUN"
}
{
  "id": "msg_UI

### Utils: Pretty Print Current Thread Content

In [25]:
print(pretty_print_thread(thread_messages))

User: "Hi Assistant!"
Assistant: "Hello! How can I assist you today?"
User: "How's the weather in Manila today?"
Assistant: "The weather in Manila today is 19°C."
User: "How's the weather and traffic condition in Cebu today?"
Assistant: "The weather in Cebu today is 28°C, and the traffic condition is described as "carmageddon levels per the usual.""



### Utils: View Run Steps

Run steps describes the 'steps' done in each Run. 

I find the Run Steps information most useful when using the 'code interpreter' tool. The intermediate steps done by the Assistant e.g. what python code it run to get to the final output is displayed here.

In [27]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id, run_id=run.id, order="asc"
)

for step in run_steps.data:
    step_details = step.step_details
    print(json.dumps(pretty_print(step_details), indent=4))

{
  "tool_calls": [
    {
      "id": "call_iaAiy3aq57J9LIJQEQsvVEt1",
      "function": {
        "arguments": "{'location': 'Cebu', 'unit': 'c'}",
        "name": "getCurrentWeather",
        "output": "28C"
      },
      "type": "function"
    },
    {
      "id": "call_ylkyAiSrLmts6Y7iihaNXKBO",
      "function": {
        "arguments": "{'location': 'Cebu'}",
        "name": "getCurrentTraffic",
        "output": "carmageddon levels per the usual"
      },
      "type": "function"
    }
  ],
  "type": "tool_calls"
}
null
{
  "message_creation": {
    "message_id": "msg_4RXAE87pt1wt5LhWspFJkdfx"
  },
  "type": "message_creation"
}
null
